#Problem Statement

#We are supposed to build a model that has the capacity to generate words on their own when we send an input of words to it and we need to classify them as UFDP(Un-fair and deceptive business practice related) or not UFDP(Un-fair and deceptive business practice related) 

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


In [48]:
train= pd.read_csv("GrievancesData_Train.csv") #loading train data
train.head()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,"Bank5279 Research dpt. \nXXXX XXXX XXXX XXXX,...",Mortgage,Closed with explanation,No,2016-01-30
1,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I had a mortgage with from XXXX of XXXX to XX...,Mortgage,Closed with explanation,Yes,2016-02-10
2,GID512413,Bank5286,State14,2016-01-19,Billing disputes,"In XXXX XXXX, I was given a {$1100.00} credit...",Credit card,Closed with monetary relief,Yes,2016-02-10
3,GID512415,Bank5279,State53,2016-01-19,"Loan servicing, payments, escrow account",Our Mortgage was sold to Bank5279 in XXXX of ...,Mortgage,Closed with explanation,No,2016-01-30
4,GID512417,Bank5286,State37,2016-01-19,"Loan modification,collection,foreclosure",We started dealing with Citi Mortgage back in...,Mortgage,Closed with explanation,No,2016-01-30


In [49]:
test= pd.read_csv("GrievancesData_Test.csv")
test.head()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
0,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",I currently have a mortgage with Flag star ba...,Mortgage,Closed with explanation,No,2016-01-30
1,GID512416,Bank5278,State16,2016-01-19,Problems when you are unable to pay,Bank5278 Auto Finance repossessed and subsequ...,Consumer Loan,Closed with explanation,No,2016-02-03
2,GID515121,Bank5372,State56,2016-03-19,"Loan servicing, payments, escrow account",We fell behind in our payments back in 2014 f...,Mortgage,Closed with explanation,No,2016-03-30
3,GID515123,Bank5372,State47,2016-03-19,"Loan servicing, payments, escrow account",My home is on XXXX parcels of land. The first...,Mortgage,Closed with explanation,No,2016-03-30
4,GID515124,Bank5310,State10,2016-03-19,Settlement process and costs,We mortgaged our home with XXXX on XXXX/XXXX/...,Mortgage,Closed with explanation,No,2016-03-30


In [21]:
train.head()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,"Bank5279 Research dpt. \nXXXX XXXX XXXX XXXX,...",Mortgage,Closed with explanation,No,2016-01-30
1,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I had a mortgage with from XXXX of XXXX to XX...,Mortgage,Closed with explanation,Yes,2016-02-10
2,GID512413,Bank5286,State14,2016-01-19,Billing disputes,"In XXXX XXXX, I was given a {$1100.00} credit...",Credit card,Closed with monetary relief,Yes,2016-02-10
3,GID512415,Bank5279,State53,2016-01-19,"Loan servicing, payments, escrow account",Our Mortgage was sold to Bank5279 in XXXX of ...,Mortgage,Closed with explanation,No,2016-01-30
4,GID512417,Bank5286,State37,2016-01-19,"Loan modification,collection,foreclosure",We started dealing with Citi Mortgage back in...,Mortgage,Closed with explanation,No,2016-01-30


In [22]:
train.shape,test.shape

((53680, 10), (27954, 10))

In [12]:
test.describe()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution
count,27954,27954,27879,27954,27954,27954,27954,27954,27954,27954
unique,27954,22,61,1168,59,24137,4,4,2,1198
top,GID382684,Bank5373,State9,2014-01-19,"Loan servicing, payments, escrow account",I was shocked when I reviewed my credit repor...,Credit card,Closed with explanation,No,2014-02-19
freq,1,8221,3839,727,4403,6,9561,23505,22524,291


In [50]:
## Calculating No of Words for train

train["word_count"]=train["GrievanceDescription"].apply(lambda x: len(str(x).split(" ")))
train[["word_count","GrievanceDescription"]].head()

,word_count,GrievanceDescription
0,169,"Bank5279 Research dpt. \nXXXX XXXX XXXX XXXX,..."
1,515,I had a mortgage with from XXXX of XXXX to XX...
2,569,"In XXXX XXXX, I was given a {$1100.00} credit..."
3,284,Our Mortgage was sold to Bank5279 in XXXX of ...
4,422,We started dealing with Citi Mortgage back in...


In [51]:
## Calculating No of Characters in each line

train["char_count"]=train["GrievanceDescription"].str.len()  # also includes spaces
train[["char_count","GrievanceDescription"]].head()

,char_count,GrievanceDescription
0,944,"Bank5279 Research dpt. \nXXXX XXXX XXXX XXXX,..."
1,2641,I had a mortgage with from XXXX of XXXX to XX...
2,2873,"In XXXX XXXX, I was given a {$1100.00} credit..."
3,1456,Our Mortgage was sold to Bank5279 in XXXX of ...
4,2281,We started dealing with Citi Mortgage back in...


In [52]:
##to check the number of stopwords in each Description

from nltk.corpus import stopwords
stop = stopwords.words('english')

train['GrievanceDescription'] = train['GrievanceDescription'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
train['GrievanceDescription'].head()


0    Bank5279 Research dpt. XXXX XXXX XXXX XXXX, CA...
1    I mortgage XXXX XXXX XXXX XXXX never missed pa...
2    In XXXX XXXX, I given {$1100.00} credit limit ...
3    Our Mortgage sold Bank5279 XXXX 2014. They bou...
4    We started dealing Citi Mortgage back XX/XX/XX...
Name: GrievanceDescription, dtype: object

In [53]:
##Number of Special Characters

train['hastags'] = train['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['GrievanceDescription','hastags']].head()


,GrievanceDescription,hastags
0,"Bank5279 Research dpt. XXXX XXXX XXXX XXXX, CA...",0
1,I mortgage XXXX XXXX XXXX XXXX never missed pa...,0
2,"In XXXX XXXX, I given {$1100.00} credit limit ...",0
3,Our Mortgage sold Bank5279 XXXX 2014. They bou...,0
4,We started dealing Citi Mortgage back XX/XX/XX...,0


In [35]:
## Removing Punctuation

data['GrievanceDescription'] = data['GrievanceDescription'].str.replace('[^\w\s]','')
data["GrievanceDescription"].head()

0    Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...
1    I had a mortgage with from XXXX of XXXX to XXX...
2    In XXXX XXXX I was given a 110000 credit limit...
3    Our Mortgage was sold to Bank5279 in XXXX of 2...
4    We started dealing with Citi Mortgage back in ...
Name: GrievanceDescription, dtype: object

In [27]:
## No of UpperCase words

data['upper'] = data['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

data[['GrievanceDescription','upper']].head(15)


,GrievanceDescription,upper
0,Bank5279 Research dpt \nXXXX XXXX XXXX XXXX C...,31
1,I had a mortgage with from XXXX of XXXX to XX...,93
2,In XXXX XXXX I was given a 110000 credit limi...,53
3,Our Mortgage was sold to Bank5279 in XXXX of ...,29
4,We started dealing with Citi Mortgage back in...,19
5,Trouble with how payments are handled My husba...,57
6,TOO BIG TO FAIL AND CARE We are in our XXXX a...,43
7,I have a car loan with Bank5397 In XXXX of 20...,7
8,Ocwen loan servicing is the servicer of my lo...,22
9,We are trying to close a new Mortgage we have...,53


In [55]:
## Rare words removal

freq = pd.Series(' '.join(data['GrievanceDescription']).split()).value_counts()[-10:]
freq

imposter              1
shortcycling          1
nknown                1
fraudulentpurposes    1
wouldcapitol          1
1002240               1
clearedmy             1
tounderstand          1
OcwenProperty         1
yearsMy               1
dtype: int64

In [56]:
##The frequency of the rare words
freq = list(freq.index)
train['GrievanceDescription'] = train['GrievanceDescription'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['GrievanceDescription'].head()

0    Bank5279 Research dpt. XXXX XXXX XXXX XXXX, CA...
1    I mortgage XXXX XXXX XXXX XXXX never missed pa...
2    In XXXX XXXX, I given {$1100.00} credit limit ...
3    Our Mortgage sold Bank5279 XXXX 2014. They bou...
4    We started dealing Citi Mortgage back XX/XX/XX...
Name: GrievanceDescription, dtype: object

In [57]:
## Removing Special Characters

train['hastags'] = train['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train[['GrievanceDescription','hastags']].head()

,GrievanceDescription,hastags
0,"Bank5279 Research dpt. XXXX XXXX XXXX XXXX, CA...",0
1,I mortgage XXXX XXXX XXXX XXXX never missed pa...,0
2,"In XXXX XXXX, I given {$1100.00} credit limit ...",0
3,Our Mortgage sold Bank5279 XXXX 2014. They bou...,0
4,We started dealing Citi Mortgage back XX/XX/XX...,0


In [30]:
from textblob import TextBlob

In [31]:
##TextBlob is used for Spelling Correction

train["GrievanceDescription"][:5].apply(lambda x: str(TextBlob(x).correct()))
train["GrievanceDescription"].head()

0     Bank5279 Research dpt. \nXXXX XXXX XXXX XXXX,...
1     I had a mortgage with from XXXX of XXXX to XX...
2     In XXXX XXXX, I was given a {$1100.00} credit...
3     Our Mortgage was sold to Bank5279 in XXXX of ...
4     We started dealing with Citi Mortgage back in...
Name: GrievanceDescription, dtype: object

In [32]:
data["GrievanceDescription"].head(10)

0    Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...
1    I had a mortgage with from XXXX of XXXX to XXX...
2    In XXXX XXXX I was given a 110000 credit limit...
3    Our Mortgage was sold to Bank5279 in XXXX of 2...
4    We started dealing with Citi Mortgage back in ...
5    Trouble with how payments are handled My husba...
6    TOO BIG TO FAIL AND CARE We are in our XXXX af...
7    I have a car loan with Bank5397 In XXXX of 201...
8    Ocwen loan servicing is the servicer of my loa...
9    We are trying to close a new Mortgage we have ...
Name: GrievanceDescription, dtype: object

In [38]:
##Text Preprocessing

##Tokenizaion

TextBlob(data["GrievanceDescription"][1]).words

WordList(['I', 'had', 'a', 'mortgage', 'with', 'from', 'XXXX', 'of', 'XXXX', 'to', 'XXXX', 'of', 'XXXX', 'and', 'never', 'missed', 'a', 'payment', 'I', 'refinanced', 'my', 'Citizens', 'Mortgage', 'through', 'XXXX', 'in', 'XXXX', 'of', 'XXXX', 'to', 'obtain', 'a', 'XXXX', 'loan', 'to', 'completely', 'renovate', 'my', 'home', 'My', 'home', 'is', 'built', 'My', 'current', 'mortgage', 'is', 'with', 'XXXX', 'for', '41000000', 'My', 'house', 'is', 'a', 'single', 'family', 'XXXX', 'in', 'XXXX', 'XXXX', 'with', 'an', 'ocean', 'view', 'XXXX', 'car', 'garage', 'and', 'an', 'additional', 'parking', 'spot', 'In', 'late', 'XXXX', 'of', 'XXXX', 'after', 'I', 'moved', 'in', 'I', 'started', 'a', 'refinancing', 'process', 'with', 'Citizens', 'to', 'lower', 'my', 'monthly', 'payment', 'by', 'close', 'to', 'or', 'at', '60000', 'because', 'I', 'would', 'no', 'longer', 'need', 'PMI', 'insurance', 'I', 'provided', 'Citizens', 'with', 'every', 'document', 'it', 'asked', 'for', 'ASAP', 'XXXX', 'for', 'XXXX', 

In [39]:
## Lemmatization( The process of converting a word to its root form from the Whole Dataset)

from textblob import Word
data["GrievanceDescription"]=data["GrievanceDescription"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data["GrievanceDescription"].head(10)

0    Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...
1    I had a mortgage with from XXXX of XXXX to XXX...
2    In XXXX XXXX I wa given a 110000 credit limit ...
3    Our Mortgage wa sold to Bank5279 in XXXX of 20...
4    We started dealing with Citi Mortgage back in ...
5    Trouble with how payment are handled My husban...
6    TOO BIG TO FAIL AND CARE We are in our XXXX af...
7    I have a car loan with Bank5397 In XXXX of 201...
8    Ocwen loan servicing is the servicer of my loa...
9    We are trying to close a new Mortgage we have ...
Name: GrievanceDescription, dtype: object

In [58]:
train.Grievance_Category.value_counts()

Loan servicing, payments, escrow account                                            8455
Loan modification,collection,foreclosure                                            6469
Billing disputes                                                                    3645
Dealing with my lender or servicer                                                  3171
Managing the loan or lease                                                          2885
Other                                                                               2142
Identity theft / Fraud / Embezzlement                                               1951
Closing/Cancelling account                                                          1599
Application, originator, mortgage broker                                            1557
Trouble during payment process                                                      1342
Problems when you are unable to pay                                                 1170
Customer service / Cu

In [59]:
train.nunique()

GrievanceID             53680
BankID                     86
State                      59
DateOfGrievance          1222
Grievance_Category         67
GrievanceDescription    43892
LineOfBusiness              4
ResolutionComments          4
Disputed                    2
DateOfResolution         1248
word_count               1008
char_count               4142
hastags                    20
dtype: int64

In [60]:
data.GrievanceDescription.value_counts()

I wa shocked when I reviewed my credit report and found late payment on the date below 30 day past due a of XXXX 2016 I am not sure how this happened I believe that I had made my payment to you when I received my statement My only thought is that my monthly statement did not get to me                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [61]:
##Finding the total number of unique values in each column

train.apply(lambda x: len(x.unique()))

GrievanceID             53680
BankID                     86
State                      60
DateOfGrievance          1222
Grievance_Category         67
GrievanceDescription    43892
LineOfBusiness              4
ResolutionComments          4
Disputed                    2
DateOfResolution         1248
word_count               1008
char_count               4142
hastags                    20
dtype: int64

In [62]:
train.columns.unique()

Index(['GrievanceID', 'BankID', 'State', 'DateOfGrievance',
       'Grievance_Category', 'GrievanceDescription', 'LineOfBusiness',
       'ResolutionComments', 'Disputed', 'DateOfResolution', 'word_count',
       'char_count', 'hastags'],
      dtype='object')

In [63]:
train.isnull().sum()

GrievanceID               0
BankID                    0
State                   121
DateOfGrievance           0
Grievance_Category        0
GrievanceDescription      0
LineOfBusiness            0
ResolutionComments        0
Disputed                  0
DateOfResolution          0
word_count                0
char_count                0
hastags                   0
dtype: int64

In [64]:
((train.isnull() | train.isna()).sum() * 100 / train.index.size).round(2)

GrievanceID             0.00
BankID                  0.00
State                   0.23
DateOfGrievance         0.00
Grievance_Category      0.00
GrievanceDescription    0.00
LineOfBusiness          0.00
ResolutionComments      0.00
Disputed                0.00
DateOfResolution        0.00
word_count              0.00
char_count              0.00
hastags                 0.00
dtype: float64

In [103]:
tar= pd.read_csv("Train.csv")

In [104]:
tar.head()

,BankID,BankGrade
0,Bank5298,satisfactory
1,Bank5421,deficient
2,Bank5326,satisfactory
3,Bank5432,outstanding
4,Bank5439,deficient


In [28]:
##Advanced Text Processing

## Generating N Grams

TextBlob(data['GrievanceDescription'][0]).ngrams(2)



[WordList(['Bank5279', 'Research']),
 WordList(['Research', 'dpt']),
 WordList(['dpt', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'CA']),
 WordList(['CA', 'XXXXXXXX']),
 WordList(['XXXXXXXX', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'name']),
 WordList(['name', 'is']),
 WordList(['is', 'XXXX']),
 WordList(['XXXX', 'XXXX']),
 WordList(['XXXX', 'I']),
 WordList(['I', 'wa']),
 WordList(['wa', 'the']),
 WordList(['the', 'customer']),
 WordList(['customer', 'for']),
 WordList(['for', 'this']),
 WordList(['this', 'loan']),
 WordList(['loan', 'I']),
 WordList(['I', 'refinanced']),
 WordList(['refinanced', 'my']),
 WordList(['my', 'home']),
 WordList(['home', 'on']),
 WordList(['on', 'XXXX']),
 WordList(['XXXX', 'The']),
 WordList(['The', 'new']),
 WordList(['new', 'loan']),
 WordList(['loan', 'funded']),
 WordList(['funded', 'on']),
 WordList(['on', 'XXXXXXXX15']),
 WordList(['XXXXXXXX15', 'I']),
 WordLi

In [29]:
##Term Frequency

#Term frequency is simply the ratio of the count of a word present in a sentence, to the length of the sentence.

#Therefore, we can generalize term frequency as:

#TF = (Number of times term T appears in the particular row) / (number of terms in that row)


tf1 = (data['GrievanceDescription'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1.head(5)

,words,tf
0,XXXX,66
1,I,25
2,wa,20
3,the,14
4,to,13


In [30]:
## Inverse Term Frequency

#The intuition behind inverse document frequency (IDF) is that a word is not of much use to us if it’s appearing in 
#all the documents.

#Therefore, the IDF of each word is the log of the ratio of the total number of rows to the number of rows in 
#which that word is present.

#IDF = log(N/n), where, N is the total number of rows and n is the number of rows in which the word was present.

for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(data.shape[0]/(len(data[data['GrievanceDescription'].str.contains(word)])))
tf1.head(5)
##The more the value of IDF, the more unique is the word.


,words,tf,idf
0,XXXX,66,0.123280
1,I,25,0.056523
2,wa,20,0.179449
3,the,14,0.033625
4,to,13,0.039616


In [31]:
##Term Frequency – Inverse Document Frequency (TF-IDF)

tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1.head(5)

,words,tf,idf,tfidf
0,XXXX,66,0.123280,8.136458
1,I,25,0.056523,1.413076
2,wa,20,0.179449,3.588988
3,the,14,0.033625,0.470755
4,to,13,0.039616,0.515004


In [32]:
##Word Embeddings
#Word Embedding is the representation of text in the form of vectors. The underlying idea here is that similar words will have 
#a minimum distance between their vectors.

from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

C:\Users\world\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


(400000, 100)

In [33]:
from gensim.models import KeyedVectors
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [34]:
model["credit"]

array([ 0.54685  , -0.22525  ,  0.095015 , -0.19641  ,  0.068867 ,
       -0.55884  , -0.66178  , -0.42628  ,  0.063782 ,  0.11867  ,
       -0.014495 ,  0.74114  , -0.51318  ,  0.17816  , -0.14786  ,
        0.072782 ,  0.39947  , -0.27028  ,  0.33603  ,  0.31553  ,
       -0.41967  ,  0.60735  , -0.048756 ,  0.15837  , -0.45795  ,
       -0.55579  , -0.91885  , -0.49961  , -0.53053  ,  0.051961 ,
        0.70411  ,  0.7985   , -0.57392  ,  0.057146 , -0.24607  ,
       -0.275    ,  0.35965  , -0.0097388, -0.21164  , -0.48524  ,
        0.81091  , -0.72492  , -0.28952  ,  0.34345  ,  0.11152  ,
        0.04223  ,  0.19924  , -0.39709  , -0.15823  , -1.236    ,
       -0.35803  ,  0.23889  , -0.28144  ,  0.51009  ,  0.46089  ,
       -2.1873   ,  0.42649  , -0.65102  ,  1.4502   ,  0.55411  ,
        0.3556   , -0.28317  , -0.46363  ,  0.13305  ,  0.4372   ,
        0.1955   ,  0.13308  ,  0.24792  ,  1.4099   , -0.39776  ,
       -0.75765  ,  0.43834  , -0.74392  , -0.32341  ,  0.2402

In [35]:
model["mortgage"]

array([ 1.2250e-01, -4.1763e-01,  3.9560e-01,  3.5478e-01, -7.3838e-01,
       -4.6325e-01, -8.1812e-01, -8.8232e-01, -2.9803e-02,  8.8155e-02,
        7.5241e-01,  6.8507e-01, -7.0571e-01,  5.3424e-01, -9.3129e-02,
        1.8605e-01, -6.4764e-01,  1.9409e-01,  9.9885e-01,  4.9778e-01,
       -5.9357e-01,  3.7057e-01,  3.5624e-02,  2.6192e-01, -5.4122e-01,
       -1.0074e+00, -9.2076e-01, -7.6004e-01, -1.3137e+00,  4.3663e-01,
        1.1632e+00,  2.4274e-01,  8.9932e-02,  7.8615e-01, -5.2177e-01,
        1.7149e-01, -6.0395e-03,  3.8522e-01,  4.9789e-01, -7.1983e-01,
        9.1180e-01, -3.7085e-01, -5.1622e-01, -2.3502e-02, -5.9027e-02,
        2.0032e-01,  5.1449e-02,  4.1327e-01, -6.8861e-02, -1.0579e+00,
       -7.4299e-01, -2.3926e-01,  4.2655e-01, -1.6381e-01,  7.4488e-01,
       -9.4614e-01, -6.8435e-02, -1.3308e+00,  1.8843e+00,  1.1070e+00,
        3.5645e-01, -3.0271e-01,  1.6110e-02, -5.2390e-01,  6.6480e-01,
        2.8964e-01,  5.6669e-04,  7.6864e-01,  8.7676e-01, -1.00

In [67]:
## Constructing TFIDF Matrix
from sklearn.feature_extraction.text import TfidfVectorizer
train.columns

Index(['GrievanceID', 'BankID', 'State', 'DateOfGrievance',
       'Grievance_Category', 'GrievanceDescription', 'LineOfBusiness',
       'ResolutionComments', 'Disputed', 'DateOfResolution', 'word_count',
       'char_count', 'hastags'],
      dtype='object')

In [68]:
list_Gr= []
for i in range(len(data["GrievanceDescription"])):
    list_Gr.append(data["GrievanceDescription"][i])

In [69]:
tfidf_transformer = TfidfVectorizer(ngram_range=(1,1), stop_words='english')

X_train_tfidf = tfidf_transformer.fit_transform(list_Gr)

X_test_tfidf = tfidf_transformer.fit_transform(list_Gr)

In [70]:
X_train_tfidf

<53680x53296 sparse matrix of type '<class 'numpy.float64'>'
	with 3544617 stored elements in Compressed Sparse Row format>

In [71]:
X_test_tfidf

<53680x53296 sparse matrix of type '<class 'numpy.float64'>'
	with 3544617 stored elements in Compressed Sparse Row format>

In [76]:
type(X_train_tfidf)

scipy.sparse.csr.csr_matrix

In [77]:
type(X_test_tfidf)

scipy.sparse.csr.csr_matrix

In [78]:
X_train_tfidf.shape


(53680, 53296)

In [79]:
X_test_tfidf.shape

(53680, 53296)

In [80]:
len(tfidf_transformer.get_feature_names())


53296

In [81]:
tfidf_transformer.get_feature_names()


['00',
 '000',
 '0000',
 '00000',
 '000000',
 '0001',
 '000balance',
 '000interest',
 '000xxxxxxxx',
 '001',
 '001informed',
 '005',
 '01',
 '010',
 '01000',
 '011',
 '012',
 '0120',
 '0125',
 '014',
 '019',
 '025',
 '03',
 '035',
 '0395',
 '05',
 '062016',
 '0625',
 '064',
 '068',
 '07',
 '078',
 '08',
 '0875',
 '09',
 '0901',
 '0945',
 '0cwen',
 '0date',
 '0f',
 '0k',
 '0monthly',
 '0n',
 '0nxxxxxxxx',
 '0r',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '1000000',
 '10000000',
 '100000000',
 '1000000000',
 '1000month',
 '1001',
 '1002',
 '1002240',
 '10029',
 '10029notifications',
 '1003',
 '10032',
 '10034',
 '10035',
 '10038',
 '1005',
 '100510',
 '1007',
 '1009',
 '100k',
 '100mo',
 '100month',
 '100th',
 '100xxxx',
 '101',
 '1011',
 '1012',
 '10125',
 '1012business',
 '10136',
 '1014',
 '1015',
 '1015days',
 '1017',
 '102',
 '1020',
 '102242',
 '1024',
 '102417',
 '102433',
 '102434',
 '102435',
 '102435102436',
 '102435mortgage',
 '102436',
 '102437',
 '102438',
 '102439',
 '10

In [74]:
from sklearn.cluster import KMeans

In [82]:
Cluster1 = KMeans(n_clusters=2)
Cluster2= KMeans(n_clusters=2)

In [83]:
predicttrain= Cluster1.fit(X_train_tfidf)

In [84]:
predicttest= Cluster2.fit(X_test_tfidf)

In [85]:
cluster_test = predicttrain.labels_.tolist()

In [86]:
cluster_test

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [113]:
cluster_tr = predicttrain.labels_.tolist()

In [114]:
cluster_tr

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [90]:
test= pd.read_csv("GrievancesData_Test.csv")

In [65]:
## Calculating No of Words for test

test["word_count"]=test["GrievanceDescription"].apply(lambda x: len(str(x).split(" ")))
test[["word_count","GrievanceDescription"]].head(15)

,word_count,GrievanceDescription
0,110,I currently have a mortgage with Flag star ba...
1,79,Bank5278 Auto Finance repossessed and subsequ...
2,374,We fell behind in our payments back in 2014 f...
3,436,My home is on XXXX parcels of land. The first...
4,90,We mortgaged our home with XXXX on XXXX/XXXX/...
5,192,I accepted a credit card off for the Barclays...
6,75,"I would like to be credited in full, of all t..."
7,548,Can't decrease my monthly payments I am trying...
8,65,I was charged a fee both when I spoke with XX...
9,660,Having problems with customer service On XXXX ...


In [91]:
## Calculating No of Characters for test

test["char_count"]=test["GrievanceDescription"].str.len()  # also includes spaces
test[["char_count","GrievanceDescription"]].head(10)

,char_count,GrievanceDescription
0,589,I currently have a mortgage with Flag star ba...
1,456,Bank5278 Auto Finance repossessed and subsequ...
2,1974,We fell behind in our payments back in 2014 f...
3,2228,My home is on XXXX parcels of land. The first...
4,562,We mortgaged our home with XXXX on XXXX/XXXX/...
5,1189,I accepted a credit card off for the Barclays...
6,403,"I would like to be credited in full, of all t..."
7,2848,Can't decrease my monthly payments I am trying...
8,346,I was charged a fee both when I spoke with XX...
9,3899,Having problems with customer service On XXXX ...


In [67]:
from nltk.corpus import stopwords
stop=stopwords.words("english")
test["stopwords"]=test["GrievanceDescription"].apply(lambda x: len([x for x in x.split() if x in stop]))
test[["GrievanceDescription","stopwords"]].head()

,GrievanceDescription,stopwords
0,I currently have a mortgage with Flag star ba...,52
1,Bank5278 Auto Finance repossessed and subsequ...,28
2,We fell behind in our payments back in 2014 f...,189
3,My home is on XXXX parcels of land. The first...,196
4,We mortgaged our home with XXXX on XXXX/XXXX/...,30


In [92]:
##Number of Special Characters for test

test['hastags'] = test['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
test[['GrievanceDescription','hastags']].head(10)

,GrievanceDescription,hastags
0,I currently have a mortgage with Flag star ba...,0
1,Bank5278 Auto Finance repossessed and subsequ...,0
2,We fell behind in our payments back in 2014 f...,0
3,My home is on XXXX parcels of land. The first...,0
4,We mortgaged our home with XXXX on XXXX/XXXX/...,0
5,I accepted a credit card off for the Barclays...,0
6,"I would like to be credited in full, of all t...",0
7,Can't decrease my monthly payments I am trying...,0
8,I was charged a fee both when I spoke with XX...,0
9,Having problems with customer service On XXXX ...,0


In [69]:
## Removing Punctuation for test

test['GrievanceDescription'] = test['GrievanceDescription'].str.replace('[^\w\s]','')
test["GrievanceDescription"].head()

0     I currently have a mortgage with Flag star ba...
1     Bank5278 Auto Finance repossessed and subsequ...
2     We fell behind in our payments back in 2014 f...
3     My home is on XXXX parcels of land The first ...
4     We mortgaged our home with XXXX on XXXXXXXXXX...
Name: GrievanceDescription, dtype: object

In [93]:
## No of UpperCase words for test

test['upper'] = test['GrievanceDescription'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

test[['GrievanceDescription','upper']].head(15)

,GrievanceDescription,upper
0,I currently have a mortgage with Flag star ba...,11
1,Bank5278 Auto Finance repossessed and subsequ...,5
2,We fell behind in our payments back in 2014 f...,11
3,My home is on XXXX parcels of land. The first...,56
4,We mortgaged our home with XXXX on XXXX/XXXX/...,21
5,I accepted a credit card off for the Barclays...,12
6,"I would like to be credited in full, of all t...",6
7,Can't decrease my monthly payments I am trying...,28
8,I was charged a fee both when I spoke with XX...,8
9,Having problems with customer service On XXXX ...,45


In [71]:
freq = pd.Series(' '.join(test['GrievanceDescription']).split()).value_counts()[-10:]
freq

Returning       1
hlpe            1
HC              1
XXXXBusiness    1
presentedto     1
Reacquired      1
onthe           1
disallowing     1
committ         1
BANKRUPCY       1
dtype: int64

In [94]:
##Finding the total number of unique values in each column in test

test.apply(lambda x: len(x.unique()))

GrievanceID             27954
BankID                     22
State                      62
DateOfGrievance          1168
Grievance_Category         59
GrievanceDescription    24137
LineOfBusiness              4
ResolutionComments          4
Disputed                    2
DateOfResolution         1198
char_count               3689
hastags                    19
upper                     258
dtype: int64

In [95]:
##Advanced Text Processing

## Generating N Grams for test

TextBlob(test['GrievanceDescription'][0]).ngrams(2)


[WordList(['I', 'currently']),
 WordList(['currently', 'have']),
 WordList(['have', 'a']),
 WordList(['a', 'mortgage']),
 WordList(['mortgage', 'with']),
 WordList(['with', 'Flag']),
 WordList(['Flag', 'star']),
 WordList(['star', 'bank']),
 WordList(['bank', 'my']),
 WordList(['my', 'XX/XX/XXXX']),
 WordList(['XX/XX/XXXX', 'payment']),
 WordList(['payment', 'was']),
 WordList(['was', 'late']),
 WordList(['late', 'reaching']),
 WordList(['reaching', 'flag-star']),
 WordList(['flag-star', 'on']),
 WordList(['on', 'XXXX/XXXX/15']),
 WordList(['XXXX/XXXX/15', 'I']),
 WordList(['I', "'ve"]),
 WordList(["'ve", 'had']),
 WordList(['had', 'this']),
 WordList(['this', 'loan']),
 WordList(['loan', 'for']),
 WordList(['for', 'over']),
 WordList(['over', 'a']),
 WordList(['a', 'year']),
 WordList(['year', 'now']),
 WordList(['now', 'and']),
 WordList(['and', 'it']),
 WordList(['it', "'s"]),
 WordList(["'s", 'the']),
 WordList(['the', 'first']),
 WordList(['first', 'time']),
 WordList(['time', 'my

In [74]:
##Term Frequency

#Term frequency is simply the ratio of the count of a word present in a sentence, to the length of the sentence.

#Therefore, we can generalize term frequency as:

#TF = (Number of times term T appears in the particular row) / (number of terms in that row)


tf2 = (test['GrievanceDescription'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf2.columns = ['words','tf']
tf2.head(5)

,words,tf
0,was,4
1,I,4
2,the,3
3,Bank5278,3
4,to,3


In [75]:
## Inverse Term Frequency

#The intuition behind inverse document frequency (IDF) is that a word is not of much use to us if it’s appearing in 
#all the documents.

#Therefore, the IDF of each word is the log of the ratio of the total number of rows to the number of rows in 
#which that word is present.

#IDF = log(N/n), where, N is the total number of rows and n is the number of rows in which the word was present.

for i,word in enumerate(tf2['words']):
  tf2.loc[i, 'idf'] = np.log(data.shape[0]/(len(test[data['GrievanceDescription'].str.contains(word)])))
  tf2.head(5)
##The more the value of IDF, the more unique is the word.

C:\Users\world\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
C:\Users\world\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
C:\Users\world\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
C:\Users\world\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


ZeroDivisionError: division by zero

In [76]:
##Term Frequency – Inverse Document Frequency (TF-IDF)

tf2['tfidf'] = tf1['tf'] * tf1['idf']
tf2.head(5)

,words,tf,idf,tfidf
0,was,4,4.557516,8.136458
1,I,4,0.713205,1.413076
2,the,3,0.688612,3.588988
3,Bank5278,3,NaN,0.470755
4,to,3,NaN,0.515004


In [97]:
cluster_tr

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [81]:
X_test_tfidf.shape

(53680, 53293)

In [116]:
cluster_tr= pd.DataFrame({'cluster_tr':cluster_tr})
s= pd.concat([cluster_tr,train_data],axis=1)


ValueError: If using all scalar values, you must pass an index

In [117]:
s.head()

,cluster_tr,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,upper,BankGrade
0,0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,Mortgage,Closed with explanation,No,2016-01-30,31,outstanding
1,0,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I had a mortgage with from XXXX of XXXX to XXX...,Mortgage,Closed with explanation,Yes,2016-02-10,93,deficient
2,0,GID512413,Bank5286,State14,2016-01-19,Billing disputes,In XXXX XXXX I wa given a 110000 credit limit ...,Credit card,Closed with monetary relief,Yes,2016-02-10,53,satisfactory
3,0,GID512415,Bank5279,State53,2016-01-19,"Loan servicing, payments, escrow account",Our Mortgage wa sold to Bank5279 in XXXX of 20...,Mortgage,Closed with explanation,No,2016-01-30,29,outstanding
4,0,GID512417,Bank5286,State37,2016-01-19,"Loan modification,collection,foreclosure",We started dealing with Citi Mortgage back in ...,Mortgage,Closed with explanation,No,2016-01-30,19,satisfactory


In [106]:
train_data.head()

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,upper,BankGrade
0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,Mortgage,Closed with explanation,No,2016-01-30,31,outstanding
1,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I had a mortgage with from XXXX of XXXX to XXX...,Mortgage,Closed with explanation,Yes,2016-02-10,93,deficient
2,GID512413,Bank5286,State14,2016-01-19,Billing disputes,In XXXX XXXX I wa given a 110000 credit limit ...,Credit card,Closed with monetary relief,Yes,2016-02-10,53,satisfactory
3,GID512415,Bank5279,State53,2016-01-19,"Loan servicing, payments, escrow account",Our Mortgage wa sold to Bank5279 in XXXX of 20...,Mortgage,Closed with explanation,No,2016-01-30,29,outstanding
4,GID512417,Bank5286,State37,2016-01-19,"Loan modification,collection,foreclosure",We started dealing with Citi Mortgage back in ...,Mortgage,Closed with explanation,No,2016-01-30,19,satisfactory


In [293]:
cluster_test=pd.DataFrame(cluster_test)
t= pd.concat([cluster_test,test],axis=1)
t

,cluster_test,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,stopwords,hastags,upper
0,1,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",I currently have a mortgage with Flag star ba...,Mortgage,Closed with explanation,No,2016-01-30,110.0,589.0,52.0,0.0,11.0
1,1,GID512416,Bank5278,State16,2016-01-19,Problems when you are unable to pay,Bank5278 Auto Finance repossessed and subsequ...,Consumer Loan,Closed with explanation,No,2016-02-03,79.0,456.0,28.0,0.0,5.0
2,1,GID515121,Bank5372,State56,2016-03-19,"Loan servicing, payments, escrow account",We fell behind in our payments back in 2014 f...,Mortgage,Closed with explanation,No,2016-03-30,374.0,1974.0,189.0,0.0,11.0
3,1,GID515123,Bank5372,State47,2016-03-19,"Loan servicing, payments, escrow account",My home is on XXXX parcels of land The first ...,Mortgage,Closed with explanation,No,2016-03-30,436.0,2228.0,196.0,0.0,56.0
4,1,GID515124,Bank5310,State10,2016-03-19,Settlement process and costs,We mortgaged our home with XXXX on XXXXXXXXXX...,Mortgage,Closed with explanation,No,2016-03-30,90.0,562.0,30.0,0.0,21.0
5,1,GID512420,Bank5259,State9,2016-01-19,Advertising and marketing,I accepted a credit card off for the Barclays...,Credit card,Closed with explanation,No,2016-01-30,192.0,1189.0,80.0,0.0,12.0
6,0,GID512426,Bank5278,State40,2016-01-19,Identity theft / Fraud / Embezzlement,I would like to be credited in full of all th...,Credit card,Closed with explanation,No,2016-03-11,75.0,403.0,31.0,0.0,6.0
7,0,GID512428,Bank5373,State43,2016-01-19,Can't repay my loan,Cant decrease my monthly payments I am trying ...,Student loan,Closed with explanation,Yes,2016-02-10,548.0,2848.0,251.0,0.0,28.0
8,1,GID512429,Bank5284,State14,2016-01-19,"Loan modification,collection,foreclosure",I was charged a fee both when I spoke with XX...,Mortgage,Closed with explanation,Yes,2016-02-10,65.0,346.0,28.0,0.0,8.0
9,1,GID512430,Bank5316,State9,2016-01-19,Dealing with my lender or servicer,Having problems with customer service On XXXX ...,Student loan,Closed with explanation,No,2016-02-25,660.0,3899.0,258.0,0.0,45.0


In [163]:
cluster_test= pd.DataFrame({'cluster_test':cluster_test})
t= pd.concat([cluster_test,test],axis=1)
t

,cluster_test,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,stopwords,hastags,upper
0,1,GID512414,Bank5334,State43,2016-01-19,"Loan servicing, payments, escrow account",I currently have a mortgage with Flag star ba...,Mortgage,Closed with explanation,No,2016-01-30,110.0,589.0,52.0,0.0,11.0
1,1,GID512416,Bank5278,State16,2016-01-19,Problems when you are unable to pay,Bank5278 Auto Finance repossessed and subsequ...,Consumer Loan,Closed with explanation,No,2016-02-03,79.0,456.0,28.0,0.0,5.0
2,1,GID515121,Bank5372,State56,2016-03-19,"Loan servicing, payments, escrow account",We fell behind in our payments back in 2014 f...,Mortgage,Closed with explanation,No,2016-03-30,374.0,1974.0,189.0,0.0,11.0
3,1,GID515123,Bank5372,State47,2016-03-19,"Loan servicing, payments, escrow account",My home is on XXXX parcels of land The first ...,Mortgage,Closed with explanation,No,2016-03-30,436.0,2228.0,196.0,0.0,56.0
4,1,GID515124,Bank5310,State10,2016-03-19,Settlement process and costs,We mortgaged our home with XXXX on XXXXXXXXXX...,Mortgage,Closed with explanation,No,2016-03-30,90.0,562.0,30.0,0.0,21.0
5,1,GID512420,Bank5259,State9,2016-01-19,Advertising and marketing,I accepted a credit card off for the Barclays...,Credit card,Closed with explanation,No,2016-01-30,192.0,1189.0,80.0,0.0,12.0
6,0,GID512426,Bank5278,State40,2016-01-19,Identity theft / Fraud / Embezzlement,I would like to be credited in full of all th...,Credit card,Closed with explanation,No,2016-03-11,75.0,403.0,31.0,0.0,6.0
7,0,GID512428,Bank5373,State43,2016-01-19,Can't repay my loan,Cant decrease my monthly payments I am trying ...,Student loan,Closed with explanation,Yes,2016-02-10,548.0,2848.0,251.0,0.0,28.0
8,1,GID512429,Bank5284,State14,2016-01-19,"Loan modification,collection,foreclosure",I was charged a fee both when I spoke with XX...,Mortgage,Closed with explanation,Yes,2016-02-10,65.0,346.0,28.0,0.0,8.0
9,1,GID512430,Bank5316,State9,2016-01-19,Dealing with my lender or servicer,Having problems with customer service On XXXX ...,Student loan,Closed with explanation,No,2016-02-25,660.0,3899.0,258.0,0.0,45.0


In [158]:
#Model Building LogisticRegression

from sklearn.linear_model import LogisticRegression

logreg_train = LogisticRegression()


In [105]:
train_data=data.merge(tar,on='BankID',how='left')
train_data

,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,upper,BankGrade
0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,Mortgage,Closed with explanation,No,2016-01-30,31,outstanding
1,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I had a mortgage with from XXXX of XXXX to XXX...,Mortgage,Closed with explanation,Yes,2016-02-10,93,deficient
2,GID512413,Bank5286,State14,2016-01-19,Billing disputes,In XXXX XXXX I wa given a 110000 credit limit ...,Credit card,Closed with monetary relief,Yes,2016-02-10,53,satisfactory
3,GID512415,Bank5279,State53,2016-01-19,"Loan servicing, payments, escrow account",Our Mortgage wa sold to Bank5279 in XXXX of 20...,Mortgage,Closed with explanation,No,2016-01-30,29,outstanding
4,GID512417,Bank5286,State37,2016-01-19,"Loan modification,collection,foreclosure",We started dealing with Citi Mortgage back in ...,Mortgage,Closed with explanation,No,2016-01-30,19,satisfactory
5,GID512418,Bank5237,State16,2016-01-19,Dealing with my lender or servicer,Trouble with how payment are handled My husban...,Student loan,Closed with explanation,No,2016-02-19,57,satisfactory
6,GID512419,Bank5257,State16,2016-01-19,"Loan modification,collection,foreclosure",TOO BIG TO FAIL AND CARE We are in our XXXX af...,Mortgage,Closed with explanation,Yes,2016-02-10,43,satisfactory
7,GID515122,Bank5397,State27,2016-03-19,Managing the loan or lease,I have a car loan with Bank5397 In XXXX of 201...,Consumer Loan,Closed with explanation,No,2016-03-30,7,satisfactory
8,GID512421,Bank5382,State14,2016-01-19,"Loan servicing, payments, escrow account",Ocwen loan servicing is the servicer of my loa...,Mortgage,Closed with explanation,Yes,2016-02-10,22,satisfactory
9,GID515125,Bank5422,State39,2016-03-19,"Loan servicing, payments, escrow account",We are trying to close a new Mortgage we have ...,Mortgage,Closed with explanation,No,2016-03-30,53,satisfactory


In [312]:
train_data.head()

,GrievanceID,BankID,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,stopwords,hastags,upper,BankGrade
0,52943,20,1199,57,2834,2,1,0,2016-01-30,169,944,68,0,31,1
1,52944,23,1199,4,14351,2,1,1,2016-02-10,515,2641,200,0,93,0
2,52945,22,1199,12,24887,1,2,1,2016-02-10,569,2873,235,0,53,2
3,52946,20,1199,43,32399,2,1,0,2016-01-30,284,1456,111,0,29,1
4,52947,22,1199,42,40631,2,1,0,2016-01-30,422,2281,189,0,19,2


In [143]:
s.dtypes

cluster_tr            int32
GrievanceID           int32
BankID                int32
State                 int32
DateOfGrievance       int32
Grievance_Category    int32
LineOfBusiness        int32
ResolutionComments    int32
Disputed              int32
DateOfResolution      int32
upper                 int32
BankGrade             int32
dtype: object

In [165]:
##here is the problem

logreg_train = LogisticRegression()

clf_lr = logreg_train.fit(s,tar)

ValueError: bad input shape (86, 2)

In [118]:
s.head()

,cluster_tr,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,upper,BankGrade
0,0,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,Mortgage,Closed with explanation,No,2016-01-30,31,outstanding
1,0,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",I had a mortgage with from XXXX of XXXX to XXX...,Mortgage,Closed with explanation,Yes,2016-02-10,93,deficient
2,0,GID512413,Bank5286,State14,2016-01-19,Billing disputes,In XXXX XXXX I wa given a 110000 credit limit ...,Credit card,Closed with monetary relief,Yes,2016-02-10,53,satisfactory
3,0,GID512415,Bank5279,State53,2016-01-19,"Loan servicing, payments, escrow account",Our Mortgage wa sold to Bank5279 in XXXX of 20...,Mortgage,Closed with explanation,No,2016-01-30,29,outstanding
4,0,GID512417,Bank5286,State37,2016-01-19,"Loan modification,collection,foreclosure",We started dealing with Citi Mortgage back in ...,Mortgage,Closed with explanation,No,2016-01-30,19,satisfactory


In [119]:
s.cluster_tr[s.cluster_tr == 0] = s.GrievanceDescription # replacing the col with the cluster values

In [123]:
s.head()

,cluster_tr,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,upper,BankGrade
0,Bank5279 Research dpt XXXX XXXX XXXX XXXX CA X...,GID512411,Bank5279,State31,2016-01-19,Settlement process and costs,Mortgage,Closed with explanation,No,2016-01-30,31,outstanding
1,I had a mortgage with from XXXX of XXXX to XXX...,GID512412,Bank5287,State26,2016-01-19,"Application, originator, mortgage broker",Mortgage,Closed with explanation,Yes,2016-02-10,93,deficient
2,In XXXX XXXX I wa given a 110000 credit limit ...,GID512413,Bank5286,State14,2016-01-19,Billing disputes,Credit card,Closed with monetary relief,Yes,2016-02-10,53,satisfactory
3,Our Mortgage wa sold to Bank5279 in XXXX of 20...,GID512415,Bank5279,State53,2016-01-19,"Loan servicing, payments, escrow account",Mortgage,Closed with explanation,No,2016-01-30,29,outstanding
4,We started dealing with Citi Mortgage back in ...,GID512417,Bank5286,State37,2016-01-19,"Loan modification,collection,foreclosure",Mortgage,Closed with explanation,No,2016-01-30,19,satisfactory


In [122]:
s= s.drop('GrievanceDescription',axis=1)

In [121]:
s.dtypes

cluster_tr              object
GrievanceID             object
BankID                  object
State                   object
DateOfGrievance         object
Grievance_Category      object
GrievanceDescription    object
LineOfBusiness          object
ResolutionComments      object
Disputed                object
DateOfResolution        object
upper                    int64
BankGrade               object
dtype: object

In [163]:
tar= pd.read_csv("Train.csv")
tar.dtypes

BankID       object
BankGrade    object
dtype: object

In [216]:
s= s.drop('State',axis=1)

In [291]:
s

,cluster_tr,BankID,Grievance_Category,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,word_count,char_count,stopwords,hastags,upper
0,1,20,57,2,1,0,1196,169,944,68,0,31
1,1,23,4,2,1,1,1207,515,2641,200,0,93
2,1,22,12,1,2,1,1207,569,2873,235,0,53
3,1,20,43,2,1,0,1196,284,1456,111,0,29
4,1,22,42,2,1,0,1196,422,2281,189,0,19
5,1,1,30,3,1,0,1216,521,2865,201,0,57
6,0,11,42,2,1,1,1207,339,1803,142,0,43
7,0,57,45,0,1,0,1246,79,413,30,0,7
8,1,51,43,2,1,1,1207,381,1988,173,0,22
9,1,65,43,2,1,0,1246,723,3766,286,0,53


In [127]:
s.dtypes

cluster_tr            object
GrievanceID           object
BankID                object
State                 object
DateOfGrievance       object
Grievance_Category    object
LineOfBusiness        object
ResolutionComments    object
Disputed              object
DateOfResolution      object
upper                  int64
BankGrade             object
dtype: object

In [124]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [141]:
s.dtypes

cluster_tr             int32
GrievanceID            int32
BankID                 int32
State                  int32
DateOfGrievance        int32
Grievance_Category     int32
LineOfBusiness         int32
ResolutionComments     int32
Disputed               int32
DateOfResolution       int32
upper                  int32
BankGrade             object
dtype: object

In [160]:
tar.head()

,BankID,BankGrade
0,25,2
1,64,0
2,31,2
3,71,1
4,75,0


In [325]:
s[['BankGrade']]=le.fit_transform(s[['BankGrade']])

In [161]:
s.head()

,cluster_tr,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,upper,BankGrade
0,698,52943,20,23,1199,57,2,1,0,1196,199,1
1,3593,52944,23,18,1199,4,2,1,1,1207,340,0
2,6102,52945,22,5,1199,12,1,2,1,1207,274,2
3,8637,52946,20,47,1199,43,2,1,0,1196,184,1
4,11037,52947,22,29,1199,42,2,1,0,1196,101,2


In [259]:
t[['GrievanceDescription']]=le.fit_transform(t[['GrievanceDescription']].astype('str'))

In [238]:
t

,cluster_test,State,Grievance_Category,GrievanceDescription,LineOfBusiness,ResolutionComments,Disputed,word_count,char_count,stopwords,hastags,upper
0,1,37,35,4835,2,1,No,110.0,589.0,52.0,0.0,11.0
1,1,7,45,964,0,1,No,79.0,456.0,28.0,0.0,5.0
2,1,50,35,15115,2,1,No,374.0,1974.0,189.0,0.0,11.0
3,1,41,35,11087,2,1,No,436.0,2228.0,196.0,0.0,56.0
4,1,1,49,15323,2,1,No,90.0,562.0,30.0,0.0,21.0
5,1,60,2,3431,1,1,No,192.0,1189.0,80.0,0.0,12.0
6,0,34,31,9678,1,1,No,75.0,403.0,31.0,0.0,6.0
7,0,37,13,16971,3,1,Yes,548.0,2848.0,251.0,0.0,28.0
8,1,5,34,9168,2,1,Yes,65.0,346.0,28.0,0.0,8.0
9,1,60,26,20246,3,1,No,660.0,3899.0,258.0,0.0,45.0


In [261]:
s[['Grievance_Category']]=le.fit_transform(s[['Grievance_Category']])

In [260]:
s['Grievance_Category']

0        57
1         4
2        12
3        43
4        42
5        30
6        42
7        45
8        43
9        43
10       43
11       46
12       46
13       43
14       42
15       25
16       45
17       35
18       43
19       45
20       62
21       22
22       38
23       42
24       43
25       42
26       43
27       46
28       43
29       15
         ..
53650    37
53651    21
53652    64
53653    64
53654    60
53655    64
53656    60
53657    64
53658    60
53659    37
53660    64
53661    61
53662    64
53663    60
53664    60
53665    64
53666    64
53667    31
53668    21
53669    37
53670    60
53671    64
53672    64
53673     7
53674    64
53675    64
53676    64
53677    64
53678    21
53679    64
Name: Grievance_Category, Length: 53680, dtype: int64

In [214]:
s[['DateOfResolution']]=le.fit_transform(s[['DateOfResolution']])

C:\Users\world\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [150]:
s

,cluster_tr,GrievanceID,BankID,State,DateOfGrievance,Grievance_Category,LineOfBusiness,ResolutionComments,Disputed,DateOfResolution,upper,BankGrade
0,698,52943,20,23,1199,57,2,1,0,1196,199,1
1,3593,52944,23,18,1199,4,2,1,1,1207,340,0
2,6102,52945,22,5,1199,12,1,2,1,1207,274,2
3,8637,52946,20,47,1199,43,2,1,0,1196,184,1
4,11037,52947,22,29,1199,42,2,1,0,1196,101,2
5,10526,52948,1,7,1199,30,3,1,0,1216,284,2
6,3,52949,11,7,1199,42,2,1,1,1207,248,2
7,3,53669,57,19,1221,45,0,1,0,1246,307,2
8,7763,52950,51,5,1199,43,2,1,1,1207,130,2
9,10738,53670,65,31,1221,43,2,1,0,1246,274,2
